# Notebook de constitution des fichiers de travail
> Ce notebook n'a pour vocation que de constituer un jeu de données de travail df_global.<br>
Il est basé sur les analyses faites dans le notebook `projDS-00-analyse.ipynb`, mais est beaucoup plus léger.<br><br>
_NB : si l'on ne veut pas recréer les fichiers `df_sentiment.csv`, `df_usertrack.csv`, `df_context.csv` il suffit de passer directement à l'éxécution des cellules "global", ou alors d'éxécuter tout le notebook après avoir passé les cellules des 3 fichiers en mode 'texte brut' (Esc + R), puis les ramener en mode 'code' si besoin plus tard (Esc + Y)_

Les choix peuvent évoluer au fil du temps et être modifiés par les intervenant(e)s sur le projet.


In [1]:
import pandas as pd
import numpy as np

import time
from datetime import timedelta
begin_time = time.time()


<a class="anchor" id="hautdepage"></a>

### Table des matières

- Fichiers d'origine => fichiers de travail
 - [sentiment_values.csv -> df_sentiment.csv (dictionnaires de sentiments)](#df_sentiment)
 - [user_track_hashtag_timestamp.csv -> df_usertrack.csv (données utilisateurs)](#df_usertrack)
 - [context_content_features.csv -> df_context.csv (spécificités des morceaux)](#df_context)


- DataFrame global à partir des fichiers de travail
 - [df_global (version complete)](#df_global)
 - [df_global_light (version light)](#df_global_light) 

## Fichiers d'origine

<a id="df_sentiment"></a>
#### Dictionnaire de sentiments : sentiment_values.csv -> df_sentiment.csv

In [2]:
start_time = time.time()

# lecture du fichier d'origine
df_sentiment = pd.read_csv('sentiment_values.csv', sep=',')
df_sentiment.dropna(axis = 0, how ='all', inplace=True)

# traitements

# --- colonnes
df_sentiment.reset_index(inplace=True)
new_cols = ['hashtag', 'vader_min', 'vader_max', 'vader_sum', 'vader_avg',
       'afinn_min', 'afinn_max', 'afinn_sum', 'afinn_avg', 'ol_min', 'ol_max',
       'ol_sum', 'ol_avg', 'ss_min', 'ss_max', 'ss_sum', 'ss_avg','X_min', 'X_max', 'X_sum', 'X_avg']
df_sentiment.columns = new_cols

# --- choix d'un dictionnaire 
# 1ere option : compléter les valeurs nulles dans l'ordre décroissant des variables nulles 'vader_avg', 'ss_avg', 'ol_avg', 'X_avg'
df_sentiment['sent_score1'] = ((df_sentiment['vader_avg'].fillna(df_sentiment['ol_avg'])).fillna(df_sentiment['X_avg'])).fillna(df_sentiment['ss_avg'])

# 2ème option : faire une moyenne de toutes les valeurs
df_sentiment['sent_score2'] = df_sentiment[['vader_avg','ol_avg','X_avg','ss_avg']].mean(numeric_only=True, axis=1)

# 3ème option : vader_avg ou la moyenne des autres variables
df_sentiment['sent_score3'] = df_sentiment['vader_avg'].fillna(df_sentiment[['ol_avg','X_avg','ss_avg']].mean(numeric_only=True, axis=1))

# écriture du fichier csv
df_sentiment = df_sentiment[['hashtag', 'sent_score2']]
df_sentiment.to_csv('df_sentiment.csv',index_label='idx_sent')

df_sentiment.info()

print("\ntps d'exécution: ",timedelta(seconds=round((time.time() - start_time),0)))
# tps d'éxécution:  0:00:00

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5290 entries, 0 to 5289
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   hashtag      5290 non-null   object 
 1   sent_score2  5290 non-null   float64
dtypes: float64(1), object(1)
memory usage: 82.8+ KB

tps d'exécution:  0:00:00


[retour en haut de page](#hautdepage)

<a id="df_usertrack"></a>
#### Données utilisateurs : user_track_hashtag_timestamp.csv -> df_usertrack.csv

In [3]:
start_time = time.time()

# lecture du fichier d'origine
df_usertrack = pd.read_csv('user_track_hashtag_timestamp.csv')
df_usertrack.dropna(axis = 0, how ='all', inplace=True)

# traitements

# --- suppression de l'entrée avec le hashtag = Nan
df_usertrack.dropna(subset=['hashtag'], inplace=True)

# --- conversion des hashtags en minuscule
df_usertrack['hashtag'] = df_usertrack['hashtag'].str.lower()

# --- hashtags trouvés ou pas dans le dictionnaire sentiment
sentiment_hashtag = df_sentiment['hashtag']
df_usertrack['hashtag_found'] = np.nan
df_usertrack['hashtag_found'] = np.where(df_usertrack['hashtag'].isin(sentiment_hashtag), 1, 0)

# --- suppression des hashtags n'apportant aucune information pertinente
df_usertrack = df_usertrack[(df_usertrack['hashtag'] != "nowpalying") &
                            (df_usertrack['hashtag'] != "nowplay") &
                            (df_usertrack['hashtag'] != "nowplayin") &
                            (df_usertrack['hashtag'] != "nowplaying") &
                            (df_usertrack['hashtag'] != "np")
                           ]

# écriture du fichier csv
df_usertrack.to_csv('df_usertrack.csv', index_label='idx_usrtrck')
df_usertrack.info(show_counts=True)

print("\ntps d'exécution: ",timedelta(seconds=round((time.time() - start_time),0)))
# tps d'éxécution:  0:02:11


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6356936 entries, 1 to 17560112
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   user_id        6356936 non-null  int64 
 1   track_id       6356936 non-null  object
 2   hashtag        6356936 non-null  object
 3   created_at     6356936 non-null  object
 4   hashtag_found  6356936 non-null  int32 
dtypes: int32(1), int64(1), object(3)
memory usage: 266.7+ MB

tps d'exécution:  0:01:08


[retour en haut de page](#hautdepage)

<a id="df_context"></a>
#### Spécificités des morceaux : context_content_features.csv -> df_context.csv

In [4]:
start_time = time.time()

# lecture du fichier d'origine
df_context = pd.read_csv('context_content_features.csv', usecols=range(0, 22))
df_context.dropna(axis = 0, how ='all', inplace=True);

# traitements
# conversion du user_id au format int + attribution de la valeur 0 aux user_id = NaN
df_context['user_id'].fillna(0, inplace=True)
df_context['user_id'] = df_context['user_id'].astype('int64')

# --- user_id, track_id et created_at qui existent ou pas dans le fichier user_track_hashtag_timestamp
usr_user_id = df_usertrack['user_id']
usr_track_id = df_usertrack['track_id']
usr_created_at = df_usertrack['created_at']

df_context['user_id_found'] = np.nan
df_context['user_track_found'] = np.nan
df_context['user_createdat_found'] = np.nan

df_context['user_id_found'] = np.where(df_context['user_id'].isin(usr_user_id), 1, 0)
df_context['user_track_found'] = np.where(df_context['track_id'].isin(usr_track_id), 1, 0)
df_context['user_createdat_found'] = np.where(df_context['created_at'].isin(usr_created_at), 1, 0)


# --- regroupement par régions
df_context['region'] = np.nan
df_context['time_zone'].fillna("", inplace=True)

reg = "America"
lst = "US|USA|America|Alaska|Hawaii|Honolulu|Atlantic Time|Arizona|Indiana|Midway Island|Mid-Atlantic|Saskatchewan|Newfoundland"
idx = df_context[df_context['time_zone'].str.contains(lst)].index
df_context.loc[idx,'region'] = reg
del lst, reg, idx

reg = "Europe"
lst = "EU|Europe|EST|Greenland|Kyiv|Belgrade|Yerevan|Warsaw|Sofia|Riga|Azores|Ljubljana|Volgograd|Sarajevo|Skopje|Bucharest|Vladivostok|Magadan|Brussels|Tallinn|Kamchatka|Novosibirsk|Krasnoyarsk|Bratislava|Zagreb|Vilnius|Yakutsk|Prague|Minsk|Copenhagen|Petersburg|Ekaterinburg|Irkutsk|Vienna|Budapest|Paris|Moscow|Lisbon|Bern|London|Amsterdam|Athens|Stockholm|Helsinki|Dublin|Madrid|Rome|Berlin|Edinburgh"
idx = df_context[df_context['time_zone'].str.contains(lst)].index
df_context.loc[idx,'region'] = reg
del lst, reg, idx

reg = "Asia"
lst = "Tokyo|Asia|Hanoi|Chennai|Mumbai|New Delhi|Singapore|Kathmandu|Urumqi|JST|Osaka|Beijing|Rangoon|Chongqing|Sapporo|Port Moresby|Solomon Is.|Jakarta|Bangkok|Guam|Kolkata|Ulaan Bataar|Kuala|Hong Kong|Taipei|Seoul|Sri Jayawardenepura"
idx = df_context[df_context['time_zone'].str.contains(lst)].index
df_context.loc[idx,'region'] = reg
del lst, reg, idx

reg = "Latine-America"
lst = "Monterrey|Lima|Bogota|Guadalajara|La Paz|Mazatlan|Chihuahua|Georgetown|Santiago|Brasilia|Quito|Buenos Aires|Caracas|Mexico City|Tijuana"
idx = df_context[df_context['time_zone'].str.contains(lst)].index
df_context.loc[idx,'region'] = reg
del lst, reg, idx

reg = "Middle-East"
lst = "Tehran|Baghdad|Cairo|Casablanca|Istanbul|Tbilisi|Jerusalem|Kabul|Karachi|Muscat|Abu Dhabi|Kuwait|Dhaka|Baku|Tashkent|Almaty|Astana|Riyadh|Islamabad"
idx = df_context[df_context['time_zone'].str.contains(lst)].index
df_context.loc[idx,'region'] = reg
del lst, reg, idx

reg = "Africa"
lst = "Pretoria|Africa|Harare|Nairobi|Cape Verde Is.|Monrovia"
idx = df_context[df_context['time_zone'].str.contains(lst)].index
df_context.loc[idx,'region'] = reg
del lst, reg, idx

reg = "Oceania"
lst = "Perth|Melbourne|Brisbane|Sydney|New Caledonia|Adelaide|Wellington|Marshall Is.|Nuku'alofa|Canberra|Auckland|Wellington|Hobart|Samoa|Fiji|Darwin"
idx = df_context[df_context['time_zone'].str.contains(lst)].index
df_context.loc[idx,'region'] = reg
del lst, reg, idx

df_context['region'].fillna("undefined", inplace=True)


# --- spécificités des morceaux
df_context['instrumentalness'].fillna(0, inplace=True)
df_context['liveness'].fillna(0, inplace=True)
df_context['speechiness'].fillna(0, inplace=True)
df_context['danceability'].fillna(0, inplace=True)
df_context['valence'].fillna(0, inplace=True)
df_context['loudness'].fillna(0, inplace=True)
df_context['tempo'].fillna(0, inplace=True)
df_context['acousticness'].fillna(0, inplace=True)
df_context['energy'].fillna(0, inplace=True)


# --- suppression des variables sans intérêt
df_context.drop(columns=['lang', 'tweet_lang', 'coordinates', 'geo', 'place', 'time_zone', 'id', 'mode', 'key'], inplace=True)


# écriture du fichier csv
df_context.to_csv('df_context.csv', index_label='idx_ctxt')
df_context.info(show_counts=True)

print("\ntps d'exécution: ",timedelta(seconds=round((time.time() - start_time),0)))
# tps d'éxécution:  0:07:06


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11614671 entries, 0 to 11614670
Data columns (total 17 columns):
 #   Column                Non-Null Count     Dtype  
---  ------                --------------     -----  
 0   instrumentalness      11614671 non-null  float64
 1   liveness              11614671 non-null  float64
 2   speechiness           11614671 non-null  float64
 3   danceability          11614671 non-null  float64
 4   valence               11614671 non-null  float64
 5   loudness              11614671 non-null  float64
 6   tempo                 11614671 non-null  float64
 7   acousticness          11614671 non-null  float64
 8   energy                11614671 non-null  float64
 9   artist_id             11614671 non-null  object 
 10  track_id              11614671 non-null  object 
 11  created_at            11614671 non-null  object 
 12  user_id               11614671 non-null  int64  
 13  user_id_found         11614671 non-null  int32  
 14  user_track_found

[retour en haut de page](#hautdepage)

In [5]:
print("\ntps d'exécution des 3 fichiers: ",timedelta(seconds=round((time.time() - begin_time),0)))


tps d'exécution des 3 fichiers:  0:07:13


## Fichier global

In [6]:
start_time = time.time()

# lecture des fichiers de travail 
df_sentiment = pd.read_csv("df_sentiment.csv")
df_context = pd.read_csv("df_context.csv")
df_usertrack = pd.read_csv("df_usertrack.csv")

print("\ntps d'exécution: ",timedelta(seconds=round((time.time() - start_time),0)))
# tps d'exécution:  0:02:13


tps d'exécution:  0:01:32


<a id="df_global"></a>
#### Version complète : df_global.csv

Enrichissement du fichier des utilisateurs `df_user (user_track_hashtag_timestamp)`
- par les sentiments : `df_sentiment (sentiment_values)` -> merge sur le hashtag
- puis par les spécificités du morceau : `df_context (context_content_features)` -> merge sur le track_id (dédoublonnage des track_id)

In [7]:
start_time = time.time()

# --- merge de user_track avec le dictionnaire de sentiment
df_global = df_usertrack.merge(right = df_sentiment, on = 'hashtag', how = 'left')

# renommage colonne score
df_global.rename(columns={"sent_score2": "sent_score"}, inplace=True)


# --- merge avec spécificités du morceau 

# dedoublonnage des morceaux (track_id)
# en suppprimant les infos déjà présentes dans df_usertrack : created_at & user_id
# et les infos non essentielles 'user_id_found', 'user_track_found', 'user_createdat_found'
df_context_cleaned = df_context.drop_duplicates(subset=['track_id']).drop(columns=['created_at', 'user_id', 'user_id_found', 'user_track_found', 'user_createdat_found'])
# merge
df_global = df_global.merge(right = df_context_cleaned, on = 'track_id', how = 'left')

# ordre des colonnes + logique
# ( + suppression des colonnes d'index des fichiers de travail : idx_sent & idx_usrtrck )
# ( + suppression des entrées spécificités + hashtag nulles )
df_global = df_global[['user_id', 'track_id', 'artist_id', 'created_at', 
                      'instrumentalness', 'liveness', 'speechiness', 'danceability', 'valence', 'loudness', 'tempo', 'acousticness', 'energy',
                      'region', 'hashtag', 'sent_score',
                       'hashtag_found'
                      ]].dropna(subset=['liveness', 'hashtag']).reset_index()

# écriture du fichier csv
df_global.to_csv('df_global.csv',index=False)
df_global.info(show_counts=True)

print("\ntps d'exécution: ",timedelta(seconds=round((time.time() - start_time),0)))
# tps d'exécution:  0:02:13

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6356931 entries, 0 to 6356930
Data columns (total 18 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   index             6356931 non-null  int64  
 1   user_id           6356931 non-null  int64  
 2   track_id          6356931 non-null  object 
 3   artist_id         6356931 non-null  object 
 4   created_at        6356931 non-null  object 
 5   instrumentalness  6356931 non-null  float64
 6   liveness          6356931 non-null  float64
 7   speechiness       6356931 non-null  float64
 8   danceability      6356931 non-null  float64
 9   valence           6356931 non-null  float64
 10  loudness          6356931 non-null  float64
 11  tempo             6356931 non-null  float64
 12  acousticness      6356931 non-null  float64
 13  energy            6356931 non-null  float64
 14  region            6356931 non-null  object 
 15  hashtag           6356931 non-null  object 
 16  

In [8]:
df_global['hashtag_found'].value_counts()

0    5925025
1     431906
Name: hashtag_found, dtype: int64

[retour en haut de page](#hautdepage)

<a id="df_global_light"></a>
#### Version "light" : df_global_light.csv

Le fichier même fichier que précédemment `df_global.csv` mais avec uniquement les entrées comportant des scores de sentiment (hashtags présents dans le fichier `df_user - user_track_hashtag_timestamp`) et en supprimant la variables devenue inutile 'hashtag_found'.

In [9]:
start_time = time.time()

df_global_light = df_global[df_global['hashtag_found'] == 1].drop(columns=['hashtag_found'])

# écriture du fichier csv
df_global_light.to_csv('df_global_light.csv',index=False)
df_global_light.info(show_counts=True)

print("\ntps d'exécution: ",timedelta(seconds=round((time.time() - start_time),0)))
# tps d'exécution:  0:00:16

<class 'pandas.core.frame.DataFrame'>
Int64Index: 431906 entries, 1 to 6356910
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   index             431906 non-null  int64  
 1   user_id           431906 non-null  int64  
 2   track_id          431906 non-null  object 
 3   artist_id         431906 non-null  object 
 4   created_at        431906 non-null  object 
 5   instrumentalness  431906 non-null  float64
 6   liveness          431906 non-null  float64
 7   speechiness       431906 non-null  float64
 8   danceability      431906 non-null  float64
 9   valence           431906 non-null  float64
 10  loudness          431906 non-null  float64
 11  tempo             431906 non-null  float64
 12  acousticness      431906 non-null  float64
 13  energy            431906 non-null  float64
 14  region            431906 non-null  object 
 15  hashtag           431906 non-null  object 
 16  sent_score        4

[retour en haut de page](#hautdepage)

In [10]:
print("\ntps d'éxécution total: ",timedelta(seconds=round((time.time() - begin_time),0)))


tps d'éxécution total:  0:11:04
